In [37]:
import subprocess

subprocess.run("word2html ./Primat_s.docx", shell=True, check=True)

CompletedProcess(args='word2html ./Primat_s.docx', returncode=0)

In [44]:
import pypandoc
output = pypandoc.convert_file('Primat_s.docx', to='html', outputfile="Primat_s.html", extra_args=['-s'])

In [107]:
#!/usr/bin/env python
from lxml import etree
import lxml.html
import pprint
import re
import os
import cssselect
from IPython.display import display, HTML
# вручную надо удалить всё после «Географические синонимы»
# придумать, как делать дрилл-даун до исходных данных
# исключать из рассмотрения Insertae sedis
from parglare import Parser, Grammar

g = Grammar.from_file("grammar.pg", ignore_case=True)
actions = {
    "Tokens":[lambda _, nodes: nodes[0] if nodes[0] else nodes[1]], # предпочитаем первый
    "Token": [lambda _, nodes: nodes[1],
              lambda _, nodes: nodes[0],
              lambda _, value: [],
              lambda _, value: []],
    "Layer": [lambda _, value: [145.0, 66.0],  #Мел
              lambda _, value: [ 66.0,  56.0], #Палеоцен
              lambda _, value: [ 56.0,  33.9], #Эоцен
              lambda _, value: [ 33.9,  23.03],
              lambda _, value: [ 23.03,  5.333],
              lambda _, value: [ 5.333,  1.806],
              lambda _, value: [ 1.806,  0.0117],
              lambda _, value: [ 0.0117, 0.0]],
    "LayerPrefix": [lambda _, value: [],
              lambda _, value: [],
              lambda _, value: []]
}
parser = Parser(g, debug=False, debug_trace=True, actions=actions)


taxons = [
    "надотряд\\b",
    "отряд\\b",
    "подотряд\\b",
    "гипотряд\\b",
    "инфраотряд\\b",
    "парвотряд\\b",
    "надсем\\b\\.",
    "сем\\b\\.",
    "подсем\\b\\.",
#    r"триба\b",
]

taxonBreadcrumbs = ["","","","","","","","",""]

tree = lxml.html.parse("Primat_s.html").getroot()

for e in tree.cssselect("span, ul, ol, li, a, font"): # TODO новый ХТМЛ формат может и не надо так чистить
    e.drop_tag()

# информация для дебага. Добавляю ссылки, по которым можно будет скроллить документ. Эти номера также попадут в таблицу с данными.
for index, e in enumerate(tree.cssselect("p, h1, h2, h3, h4, h5, h6, table tr td")):
    anchorStr = "debug"+str(index)
    anchorElement = etree.Element("a", name="debug"+str(index))
    anchorElement.set("class", "debug")
    anchorElement.tail = e.text
    e.text = None
    e.insert(0, anchorElement)# = anchorStr + e.text if e.text else anchorStr



In [109]:
outCSV = "\t".join(taxons + ["род", "period", "debug"])
#print(outCSV)

for e in tree.cssselect("p, h1, h2, h3, h4, h5, h6, table tr"):
    #line = re.sub(r"\t", " ", line)
    if e.tag == "tr" and len(e)==3: # trying to grab the period
        # считываем название рода. Берём первое слово со знаками
        genusStr = e[0].text_content()
        genusStr = re.sub(r"\n|\t", r" ", genusStr)
        if re.match(r"^(\S+).*", genusStr, flags=re.DOTALL):
            genusStr = re.sub(r"^(\S+\s+\S*).*", r"\1", genusStr) #re.DOTALL чтобы обрезать многострочный текст до конца
            if genusStr:
                # считываем и парсим период
                periodStr = e[1].text_content()
                periodStr = re.sub(r"\n|\t", " ", periodStr)
                periodStr = re.sub(r"\(.*\)", r"", periodStr)
                periodStr = re.sub(r"(\d),(\d)", r"\1.\2", periodStr) # changing digit separator: 0,1 to 0.1
                period = parser.parse(periodStr)
                #print("genus", genusStr)
                #print("period", period)
#                if period: # если период распарсился TODO не только в этом случае выводить
                debugStr = e[0][0].get("name")
                outLine = "\t".join(taxonBreadcrumbs+[str(genusStr), str(period), str(debugStr)])
                print(outLine)
                outCSV = outCSV + os.linesep + outLine
                pass
    else: # if it's p, h1 and so on
        paragraphStr = e.text_content()
        paragraphStr = re.sub(r"\n|\t", r" ", paragraphStr)
        for index, t in enumerate(taxons):
            #match = re.sub(r"^\s*("+t+".*)", "* "*index+r"* \1", line, flags=re.IGNORECASE)
            #print(paragraphStr)
            if re.search(r"^\s*("+t+".*)", paragraphStr, flags=re.IGNORECASE):
                # Когда спускаемся по цепочке таксонов, запоминаем все, что выше.
                taxon = re.sub(r"^\s*"+t+r"\s*\b(\w+)\b.*", r"\1", paragraphStr, flags=re.IGNORECASE)
                #print(taxon)
                taxonBreadcrumbs[index] = taxon
                # А если надо подняться наверх, Пр. на уровень семейства, меняем название семейства, всё, что ниже — стираем.
                # Если проваливаемся сразу на несколько уровней, то они остаются пустыми.
                taxonBreadcrumbs = taxonBreadcrumbs[:index+1]
                taxonBreadcrumbs.extend([""]*(len(taxons)-index-1))
                #print(taxonBreadcrumbs)
                break;


#for e in tree.cssselect("table>tr"):
#    if(len(e)==3):
#        pass
        #e[0].text = "111"

#display(HTML(str(lxml.html.tostring(tree)))) работает, но выводить в ХТМЛ кучу \n\t
        
out = open('out.tsv', 'w')
#result = lxml.html.tostring(tree, pretty_print=True, method="html")
out.write(outCSV)
out.close()

out = open('Primat_s_debug.html', 'w')
result = lxml.html.tostring(tree, pretty_print=True, method="html", encoding='unicode')
#print(result[:30000])
out.write(result)
out.close()

ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Anathana Lyon,	None	debug82
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Anathana ellioti	None	debug87
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Anathana ellioti	None	debug96
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Anathana ellioti	None	debug99
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Anathana ellioti	None	debug102
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	*	[]	debug105
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Dendrogale Gray,	None	debug108
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Dendrogale melanura	None	debug113
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Dendrogale melanura	None	debug118
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Dendrogale melanura	None	debug121
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	–	[]	debug124
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Dendrogale murina	None	debug127
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	*	[]	debug133
ARCHONTA	SCANDENTIA						Tupaiidae	Tupaiinae	Tupaia Raffles

ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Galeopterus Thomas,	None	debug617
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Galeopterus variegatus	None	debug623
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		***	[]	debug629
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Dermotherium Ducrocq,	[56.0, 33.9]	debug632
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Dermotherium chimaera	[33.9, 23.03]	debug635
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Dermotherium major	[56.0, 33.9]	debug643
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Cynocephalidae		Dermotherium sp.	[56.0, 33.9]	debug646
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Mixodectidae		Mixodectes Cope,	[66.0, 56.0]	debug656
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Mixodectidae		Mixodectes malaris	[66.0, 56.0]	debug665
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Mixodectidae		Mixodectes pungens	[66.0, 56.0]	debug670
ARCHONTA	DERMOPTERA					PLAGIOMENOIDEA	Mixodectidae		*	[]	debug678
ARCHONTA	DER

ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis anceps	[66.0, 56.0]	debug1241
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis churchilli	[66.0, 56.0]	debug1262
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis cookei	[66.0, 56.0]	debug1265
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis dubius	[66.0, 56.0]	debug1272
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis fodinatus	[66.0, 56.0]	debug1284
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis gingerichi	[66.0, 56.0]	debug1294
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis insignis	[66.0, 56.0]	debug1297
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Plesiadapoidea	Plesiadapidae	Plesiadapinae	Plesiadapis praecursor	[66.0,

ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Microsyopinae	Microsyops latidens	None	debug1870
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Microsyopinae	Microsyops scottianus	None	debug1880
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Microsyopinae	???Microsyops alfi	None	debug1886
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Uintasoricinae	Choctawius foxi	[56.0, 33.9]	debug1900
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Uintasoricinae	*	[]	debug1903
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Uintasoricinae	Navajovius Matthew	[66.0, 56.0]	debug1906
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Uintasoricinae	Navajovius kohlhaasae	[66.0, 56.0]	debug1913
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomyoidea	Microsyopidae	Uintasoricinae	Navajovius? mckennai	[56.0, 33.9]	debug1923
ARCHONTA	PLESIADAPIFORMES	TUPAIOIDEA				Paromomy

ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Adapidae	Djebelemurinae	*	[]	debug2618
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Adapidae	Djebelemurinae	Djebelemur martinezi	[56.0, 33.9]	debug2621
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		Azibius trerki	[56.0, 33.9]	debug2634
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		(=Dralestes hammadaensis	[56.0, 33.9]	debug2645
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		=Tabelia hammadae	[56.0, 33.9]	debug2648
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		Azibius sp.	[56.0, 33.9]	debug2656
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		-	[]	debug2663
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		Algeripithecus minutus	[56.0, 33.9]	debug2666
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Azibiidae		(=Dralestes (частичный	[56.0, 33.9]	debug2681
ARCHONTA	PRIMATES	STREPSIRRHINI		ADAPIFORMES			Notharctidae	Asiadapinae	Asiadapis cambayensis	[56.0, 33.9]	debug2692
ARCHONT

...

ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	Kalepithecus songhorensis	[23.03, 5.333]	debug16660
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	*	[]	debug16667
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	Kamoyapithecus hamiltoni	[33.9, 23.03]	debug16670
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	*	[]	debug16683
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	Limnopithecus Hopwood,	[23.03, 5.333]	debug16686
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	Limnopithecus evansi	[23.03, 5.333]	debug16694
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Proconsulidae	Proconsulinae	Limnopithecus legetet	[23.03

ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	[23.03, 5.333]	debug17493
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	[23.03, 5.333]	debug17498
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	[23.03, 5.333]	debug17501
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	None	debug17504
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	None	debug17510
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	None	debug17516
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	None	debug17521
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	incertae		Gen. et	None	debug17528
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoi

ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		Dryopithecus fontani	[23.03, 5.333]	debug18008
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		-	[]	debug18020
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		Dryopithecus laietanus	None	debug18023
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		?=Dryopithecus crusafonti	[23.03, 5.333]	debug18030
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		??=Dryopithecus piveteaui	None	debug18033
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		?=Rahonapithecus sabadellensis	None	debug18039
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		?=Sivapithecus occidentalis	None	debug18044
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae		?Dryopith

ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	=Gorilla gorilla	None	debug18428
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	=Gorilla gorilla	None	debug18431
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	Gorilla gorilla	None	debug18434
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	Когда определяется	[]	debug18443
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	Gorilla beringei	None	debug18446
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	Gorilla beringei	None	debug18451
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	Gorilla beringei	[]	debug18457
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Pongidae	Ponginae	*	[]

ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	Australopithecus sicilianus	None	debug18890
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	Australopithecus africanus	None	debug18893
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	=Australopithecus prometheus	[]	debug18902
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	=Australopithecus transvaalensis	[]	debug18905
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	Australopithecus garhi	None	debug18911
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithecinae	Australopithecus sediba	None	debug18914
ARCHONTA	PRIMATES	HAPLORRHINI	ANTHROPOIDEA	CATARRHINI	EUCATARRHINI	Hominoidea	Hominidae	Australopithec

In [135]:






















from parglare import Parser, Grammar

grammar = r"""
Text: Tokens | EMPTY;
Tokens: Token Tokens?;
Token: LayerPrefix Layer {3}
     | Layer {2}
     | Skip {1} 
     | LayerPrefix {1};
Layer: Mel_fixme
     | Paleocene
     | Eocene 
     | Oligocene 
     | Miocene 
     | Pliocene 
     | Pleistocene 
     | Holocene
     ;
LayerPrefix: Upper 
     | Middle 
     | Lower;

terminals
Mel_fixme: "Мел";
Paleocene: "Палеоцен";
Eocene: "Эоцен";
Oligocene: "Олигоцен";
Miocene: "Миоцен";
Pliocene: "Плиоцен";
Pleistocene: "Плейстоцен";
Holocene: "Голоцен";

Upper: "в.";
Middle: "ср.";
Lower: "н.";

Skip: /./;
"""

actions = {
    "Tokens":[lambda _, nodes: nodes[0] if nodes[0] else nodes[1]], # предпочитаем первый
    "Token": [lambda _, nodes: nodes[1],
              lambda _, nodes: nodes[0],
              lambda _, value: [],
              lambda _, value: []],
    "Layer": [lambda _, value: [145.0, 66.0],  #Мел
              lambda _, value: [ 66.0,  56.0], #Палеоцен
              lambda _, value: [ 56.0,  33.9], #Эоцен
              lambda _, value: [ 33.9,  23.03],
              lambda _, value: [ 23.03,  5.333],
              lambda _, value: [ 5.333,  1.806],
              lambda _, value: [ 1.806,  0.0117],
              lambda _, value: [ 0.0117, 0.0]],
    "LayerPrefix": [lambda _, value: [],
              lambda _, value: [],
              lambda _, value: []]
}

g = Grammar.from_string(grammar, ignore_case=True)
#parser = Parser(g, debug=False, debug_trace=True)
parser = Parser(g, debug=False, debug_trace=True, actions=actions)

#result = parser.parse("     - в.палеоцен-н.-ср.-???в.эоцен")
result = parser.parse("")
print(result)

[]
